# Package Installation for Geodata Processing

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Test the Installation

In [ ]:
import importlib
def importPackages(modules):
    for library in modules:
        try:
            if library == 'gdal':
                exec("from osgeo import gdal")
                print("Package {} successfully imported!".format(library))
            else:
                exec("from {module} import *".format(module=library))
            print("Package {} successfully imported!".format(library))
        except Exception as e:
            print("\n\n E R R O R !!! With Package {}".format(library))
            print(e)
    #print(dir()) # Exactly working as thought
pckList = []
pckList.append('gdal')
pckList.append('psycopg2')
pckList.append('shapely')
pckList.append('fiona')
pckList.append('rasterio')
pckList.append('geopandas')
pckList.append('folium')
pckList.append('json')
pckList.append('os')
pckList.append('sys')

importPackages(pckList)

## Test Database connection to PostgreSQL/PostGIS

<font color='red'>You need to set the correct credentials (user & password) first!</font>

In [ ]:
import psycopg2
def postgres_test(dbn,h,u,pwd,p):
    try:
        conn = psycopg2.connect("dbname={} host={} user={} password={} port={}".format(dbn,h,u,pwd,p))
        conn.close()
        return True
    except:
        return False

database = "casdb2023"
host = "ikgpgis.ethz.ch"
usr = "xxx"
pwd = "xxx"
port = "5432"
print(postgres_test(database,host,usr,pwd,port))


## Test Reading Data

Copy from one format (Esri Shape) to another (MapInfo) using commandline tool ```ogr2ogr```

In [ ]:
import os
os.system('ogr2ogr -f "MapInfo File" Data/Gemeinden_Solothurn.tab Data/Gemeinden_Solothurn.shp')
os.system('ogr2ogr -f "GeoJSON"  -t_srs "EPSG:4326" Data/Gemeinden_SolothurnWGS84.json Data/Gemeinden_Solothurn.shp')


Read some data..

You should see something like:
```
<osgeo.ogr.DataSource; proxy of <Swig Object of type 'OGRDataSourceShadow *' at 0x7fab289df960> >
Gemeinden_Solothurn
109
Der Datensatz 'Gemeinden_Solothurn' hat 109 Gemeinden
```

In [ ]:
from osgeo import ogr
drv = ogr.GetDriverByName("ESRI Shapefile")
path2ds = "Data/Gemeinden_Solothurn.shp"
datasource = drv.Open(path2ds)
print(datasource)

mylayer = datasource.GetLayer(0)
print(mylayer.GetName())

ftrcnt = mylayer.GetFeatureCount()
print(ftrcnt)

print("Der Datensatz '%s' hat %i Gemeinden" %(mylayer.GetName(),int(ftrcnt)))

## Rasterdaten


You should see something like:
```
Anzahl Spalten: 5800
Anzahl Zeilen: 4800
Anzahl Baender: 3
```

In [ ]:
from osgeo import gdal

fn = 'Data/ortho14_5m_rgb_solothurn.tif'
ds = gdal.Open(fn)
if ds is None:
    print ('Datensatz %s konnte nicht geöffnet werden!' %fn)
    sys.exit()

#Dimension des Rasterbildes
cols = ds.RasterXSize
rows = ds.RasterYSize
bands = ds.RasterCount

print ("Anzahl Spalten: %d" %cols)
print ("Anzahl Zeilen: %d" %rows)
print ("Anzahl Baender: %d" %bands)

## Commandline commands


You should see something like:
```
command to run: gdaldem slope ../Data/Elevation_raster.tif ../Data/Ele_slope.tif -s 10000
0...10...20...30...40...50...60...70...80...90...100 - done.
```

In [ ]:
imagefilename = "Data/Elevation_raster.tif"
path = "Data/"

sloapcommand = 'gdaldem slope %s %sEle_slope.tif -s 10000' %(imagefilename,path)
print ("command to run: %s" %sloapcommand)   #gdaldem slope input_dem output_slope_map

os.system(sloapcommand)

## Geowebservices

You should see something like:
```
Successfully downloaded resource http://wms.geo.admin.ch/?SERVICE=WMS&REQUEST=GetMap&VERSION=1.3.0&LAYERS=ch.bfs.arealstatistik-1985-04&STYLES=default&CRS=EPSG:21781&BBOX=550000,60000,660000,140000&WIDTH=800&HEIGHT=582&FORMAT=image/png
```

In [ ]:
# -*- coding: utf-8 -*-
import os, shutil, sys
import urllib.request
from osgeo import gdal
from osgeo.gdalconst import *

def download(url, dest, fileName=None):
    try:
        r= urllib.request.urlopen(url)
        fileName = os.path.join(dest, fileName)
        print(f"Download stored in {fileName}.")
        with open(fileName, 'wb') as f:
            shutil.copyfileobj(r,f)
        r.close()
        print("Successfully downloaded resource {}".format(url))
    except:
        print("ERROR Downloading resource {}".format(url))

path2save2 = "Data/" #Zielpfad
wmsfile = "wms.gif"
wmslink = "https://wms.geo.admin.ch/?SERVICE=WMS&REQUEST=GetMap&VERSION=1.3.0&LAYERS=ch.bafu.bundesinventare-bln&STYLES=default&CRS=EPSG:2056&BBOX=2550000,1060000,2660000,1140000&WIDTH=800&HEIGHT=582&FORMAT=image/png"
download(wmslink,path2save2,wmsfile)
    

## Shapely

You should see something like:
```
Gemeinde Rohr hat folgenden Zentroid: (638811.067724, 251323.787658) und folgende Flaeche 2229578.988870m2
```

In [ ]:
from osgeo import ogr
import shapely.wkt

shapefile = ogr.Open("Data/Gemeinden_Solothurn.shp")
if shapefile is None:
    print ("Datensatz konnte nicht geoeffnet werden.\n")
    sys.exit()

layer = shapefile.GetLayer(0)

#Gemeindegeometry extrahieren:
geometry = None
cnt = 0
for feature in layer:
    while cnt < 1:
        #Extract Gemeinde-Name
        gemname = feature.GetField("gmde_name")
        #Get Geometry (Polygon)
        gemgeometry = feature.GetGeometryRef()
        #"Convert" Geometry to shapely-geometry
        gemgeomaswkt = gemgeometry.ExportToWkt()
        shapelypolygon = shapely.wkt.loads(gemgeomaswkt)
        #Extract Centroid
        centroid_point = shapelypolygon.centroid
        x=centroid_point.x
        y=centroid_point.y
        area = shapelypolygon.area
        #Printout Information
        print ("Gemeinde %s hat folgenden Zentroid: (%f, %f) und folgende Flaeche %fm2" %(gemname, x, y, area))
        cnt += 1

## Fiona

You should see something like:
```
Anzahl Datensätze: 109 
Format: ESRI Shapefile
Geo-Referenzsystem: {'proj': 'somerc', 'lat_0': 46.9524055555556, 'lon_0': 7.43958333333333, 'k_0': 1, 'x_0': 600000, 'y_0': 200000, 'ellps': 'bessel', 'units': 'm', 'no_defs': True}
Ausdehnung: (592560.389, 213702.99, 644759.038, 261329.631)
```

In [ ]:
import fiona

c = fiona.open('Data/Gemeinden_Solothurn.shp', 'r')
print("Anzahl Datensätze: %i " %len(list(c)))
print("Format: %s" %c.driver)
print("Geo-Referenzsystem: %s" %c.crs)
print("Ausdehnung: %s" %str(c.bounds))



## Folium

An interactive map located at EHT ZH Hönggerberg with a pin should appear.

In [ ]:
import folium, json
m = folium.Map(location=[47.40875, 8.50778], zoom_start=17)

folium.Marker(
    location=[47.40875, 8.50778],
    popup="ETH",
    icon=folium.Icon(color="red", icon="info-sign"),
).add_to(m)
m